# Load Generator for Placing Orders

**Try Endpoint Responce**

In [14]:
import requests

/Users/viktoriiavlasenko/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [32]:
BE_HOSTNAME = 'http://localhost:8080'

In [33]:
url = f'{BE_HOSTNAME}/api/v1/orders'
order = {
  "items": [
    { "productId": "B002PD61Y4", "quantity": 2 },
    { "productId": "B002SZEOLG", "quantity": 1 }
  ]
}

r = requests.post(url, json = order)

print(f"Order is placed with http status {r.status_code}")

Order is placed with http status 201


## Retrieve Products Ids from Inventory Database

**Load Custom MySQl Connector**

In [1]:
from scripts.connector import get_connector

In [2]:
cnx = get_connector()
cursor = cnx.cursor()

cursor.execute("select distinct(id) from product;")

product_ids = list()

for id in cursor.fetchall():
    product_ids.append(id[0])

cursor.close()
cnx.close()

print(f"Product ids sample: {product_ids[:5]}")
print(f"Total unique ids: {len(product_ids)}")

Product ids sample: ['B002PD61Y4', 'B002SZEOLG', 'B003B00484', 'B003L62T7W', 'B004IO5BMQ']
Total unique ids: 1351


In [3]:
product_ids[0]

'B002PD61Y4'

## Check Inventory

In [28]:
def check_inventory(hours): 
    url = f'{BE_HOSTNAME}/api/v1/inventory/changes?hours={hours}'
    r = requests.get(url)
    changes = r.json()
    print(f"Inventory change in last {hours} hours: {changes}")
    return changes

In [29]:
check_inventory(1)

Inventory change in last 1 hours: [{'productId': 'B07989VV5K', 'soldLastHours': 2, 'currentStock': 49998}]


[{'productId': 'B07989VV5K', 'soldLastHours': 2, 'currentStock': 49998}]

## Generate Orders Load

In [9]:
import random

**Generate order**

In [11]:
def place_order():
    order_items = []
    for p_id in random.sample(product_ids, k=random.randint(1, 3)):
        order_items.append({
            "productId": p_id,
            "quantity": random.randint(1, 3)
        })

    payload = {"items": order_items}

    r = requests.post(f"{BE_HOSTNAME}/api/v1/orders", json=payload)
    return payload, r.status_code

In [17]:
order_items, order_status = place_order()

print(f"Order {order_items} is placed with http status {order_status}")

Order {'items': [{'productId': 'B07989VV5K', 'quantity': 2}]} is placed with http status 201


**Create Load**

In [4]:
from concurrent.futures import ThreadPoolExecutor

In [12]:
def run_load(concurrency, requests_count):
    with ThreadPoolExecutor(max_workers=concurrency) as executor:
        futures = [executor.submit(place_order) for _ in range(requests_count)]
        for f in futures:
            f.result()

In [30]:
run_load(concurrency=50, requests_count=50)

**Check Inventory after Generated Load**

In [31]:
check_inventory(1)

Inventory change in last 1 hours: [{'productId': 'B095JQVC7N', 'soldLastHours': 3, 'currentStock': 49997}, {'productId': 'B08CT62BM1', 'soldLastHours': 1, 'currentStock': 49999}, {'productId': 'B00R1P3B4O', 'soldLastHours': 3, 'currentStock': 49997}, {'productId': 'B00ABMASXG', 'soldLastHours': 1, 'currentStock': 49999}, {'productId': 'B0BMVWKZ8G', 'soldLastHours': 3, 'currentStock': 49997}, {'productId': 'B08YD264ZS', 'soldLastHours': 2, 'currentStock': 49998}, {'productId': 'B08D75R3Z1', 'soldLastHours': 1, 'currentStock': 49999}, {'productId': 'B097R45BH8', 'soldLastHours': 1, 'currentStock': 49999}, {'productId': 'B09BW334ML', 'soldLastHours': 3, 'currentStock': 49997}, {'productId': 'B01LYLJ99X', 'soldLastHours': 2, 'currentStock': 49998}, {'productId': 'B0162K34H2', 'soldLastHours': 2, 'currentStock': 49998}, {'productId': 'B09BF8JBWX', 'soldLastHours': 3, 'currentStock': 49997}, {'productId': 'B07JB2Y4SR', 'soldLastHours': 1, 'currentStock': 49999}, {'productId': 'B08G1RW2Q3', '

[{'productId': 'B095JQVC7N', 'soldLastHours': 3, 'currentStock': 49997},
 {'productId': 'B08CT62BM1', 'soldLastHours': 1, 'currentStock': 49999},
 {'productId': 'B00R1P3B4O', 'soldLastHours': 3, 'currentStock': 49997},
 {'productId': 'B00ABMASXG', 'soldLastHours': 1, 'currentStock': 49999},
 {'productId': 'B0BMVWKZ8G', 'soldLastHours': 3, 'currentStock': 49997},
 {'productId': 'B08YD264ZS', 'soldLastHours': 2, 'currentStock': 49998},
 {'productId': 'B08D75R3Z1', 'soldLastHours': 1, 'currentStock': 49999},
 {'productId': 'B097R45BH8', 'soldLastHours': 1, 'currentStock': 49999},
 {'productId': 'B09BW334ML', 'soldLastHours': 3, 'currentStock': 49997},
 {'productId': 'B01LYLJ99X', 'soldLastHours': 2, 'currentStock': 49998},
 {'productId': 'B0162K34H2', 'soldLastHours': 2, 'currentStock': 49998},
 {'productId': 'B09BF8JBWX', 'soldLastHours': 3, 'currentStock': 49997},
 {'productId': 'B07JB2Y4SR', 'soldLastHours': 1, 'currentStock': 49999},
 {'productId': 'B08G1RW2Q3', 'soldLastHours': 1, 'c